In [8]:
#IMPORTS
import requests
import json
from google.colab import userdata
import pandas as pd
from IPython.display import display,HTML
import gradio as gr
#DATAFRAME
multi_prompt=pd.DataFrame(columns=['Prompt Style','Prompt','AI Reply','Token Used'])
#DEFINES FUNCTION
def multi_style(prompt,prompt_style,prompt_length):
 prompt_s={"news":"summarize this text in a formal journalistic news tone",
               "casual":"summarize this text in a friendly ,easygoing tone",
               "tweet": "Summarize this text as a catchy tweet under 280 characters."}
 prompt_l={"short": "Make it concise, 1–2 sentences.",
        "medium": "Keep it around 4–5 sentences.",
        "long": "Give a detailed summary in about 8–10 sentences."}
 if prompt_length=="short":
  prompt_le=prompt_l["short"]
 elif prompt_length=="medium":
  prompt_le=prompt_l["medium"]
 elif prompt_length=="long":
  prompt_le=prompt_l["long"]

 if prompt_style=="news":
  prompt_st=prompt_s["news"]
 elif prompt_style=="casual":
  prompt_st=prompt_s["casual"]
 elif prompt_style=="tweet":
  prompt_st=prompt_s["tweet"]
 data={"model":"mistralai/mistral-small-3.2-24b-instruct",
      "messages":[{"role":"system","content":"you are a helpful summarizer assistant"},
                  {"role":"user","content":f"{prompt_st},{prompt_le}.\n\n {prompt}"}],"max_tokens":80}


 global multi_prompt
 #OPENROUTER URL
 url="https://openrouter.ai/api/v1/chat/completions"
 # MY SECRET KEY
 api_key=userdata.get('MY_API')
 #HEADERS
 headers={
    "Content-Type":"application/json",
    "Authorization":f"Bearer {api_key}"
 }
 #SEND REQUEST
 response=requests.post(url,headers=headers,json=data)


 result=response.json()
 reply=result['choices'][0]["message"]["content"]
 print(f"\n {prompt_style} ::: {reply}")
 token_used=result['usage']['total_tokens']
 new_row={'Prompt Style':prompt_style,'Prompt':prompt,'AI Reply':reply,'Token Used':token_used}
 #CONTENTS COPIED IN DATAFRAME
 multi_prompt=pd.concat([multi_prompt,pd.DataFrame([new_row])],ignore_index=True)
 multi_prompt.to_csv('multi_prompt.csv',index=False)

 #RETURN FORMATTED RESULT FOR DISPLAY (FOR GRADIO)
 return f"**{prompt_style.upper()} SUMMARY \n\n {reply} \n\n Token Used:{token_used}"

# FOR DOWNLOAD CSV
def download_csv():
  return "multi_prompt.csv"

#Create layout with extra buttons and styles
with gr.Blocks(theme=gr.themes.Citrus()) as app:
  gr.Markdown("🖼## Multi Prompt Summarizer##🖼")
  gr.Markdown("Summarize any text in **NEWS** , **CASUAL** or **TWEET** tone- In short, medium or long format using the Mistral model via OpenRouter")
  #user input section
  with gr.Row():
    prompt= gr.Textbox(label="Enter your text",lines=5,placeholder="Paste or type your paragraph here...")
    style=gr.Dropdown(["news","casual","tweet"],label="Select Tone")
    length=gr.Dropdown(["short","medium","long"],label="Select Length")
 #Action + Output Section
  submit=gr.Button("Summarize now")
  csv_btn=gr.Button("Download CSV")
  output=gr.Markdown(label="AI Reply")
  #Connecting buttons to functions
  submit.click(fn=multi_style,inputs=[prompt,style,length],outputs=output)
  csv_btn.click(fn=download_csv, outputs=gr.File())
  #Footer section
  gr.Markdown("-----")
  gr.Markdown("Made with 💖 by Madiha Atif | [GitHub Repo](https://github.com/mahambilalandahaan/Advanced_multisummarizer_miniweb.git)")
#Launch the app

app.launch(share=True ,debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f431533cb83e50a956.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
